## Winescrapper - Produção

Versão de produção do script que vasculha o site da wine

- Procura todo o catálogo de Vinhos tintos e brancos
- Extrai os seguintes atributos da lista de vinhos
..* Nome do Vinho
..* Tipo
..* Link para o site da wine
..* Descrição do vinho
..* Nota da Wine
..* Numero de avaliações da wine
..* Preço para não sócios do clube wine
..* Preço cheio
..* Desconto aplicado

- Para cada vinho, o sistema ainda extari:
..* Uva
..* Classificação: seco, meio seco, etc.

Retorna um Dataframe


### 1. Imports


In [1]:
# Webscrapper do Site da Wine
# Cabeçalhos e Imports

from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import time

URL_RAIZ="https://www.wine.com.br"
URL_BASE = URL_RAIZ+"/vinhos/{0}/cVINHOS-atTIPO_{1}-p{2}.html"


HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'),
    'Connection':'close'
}


### 2. Funções

In [6]:
# Funções

def get_soup(page_url, retry_count=0):
    
    session= requests.Session()
    session.trust_env = False
    
    print("Iniciando session_get")
    
    try:
        response = session.get(page_url, headers=HEADERS)
        print("\nScraping page:",page_url," = ", response)
        soup = bs(response.content, 'html.parser')
        return soup
    except:
        print("Get falhou")
        retry_count += 1
        if retry_count <= 3:
            print("retry get")
            session = requests.Session()
            get_soup(page_url, retry_count)
        else:
            raise
   

def get_num_pages(soup):
    
    pages_list = soup.find('div', class_='Pagination').ul.find_all('li')
    num_pages = int(pages_list[len(pages_list)-1].a.text) + 1
    
    return num_pages

def scrape_technical_details(wine_url):
    wine_soup= get_soup(wine_url)
    
    technical_details = wine_soup.find('div', class_='TechnicalDetails')
    tipo_e_uva = technical_details.find('div', class_='TechnicalDetails-description--grape')
    uva = tipo_e_uva.find('div', class_="Right").dd.text

    classificacao_container = technical_details.find('div', class_='TechnicalDetails-description--classification')
    classificacao = classificacao_container.find('div', class_="Right").dd.text

    print('{}: {}'.format(classificacao, uva))    
    
    return uva, classificacao
    
def scrape_soup(soup):
    
    list_name=[]
    list_link, list_country, list_type, list_description= [], [], [], []
    list_evaluation, list_rating_count= [], []
    list_lowest_price, list_full_price, list_discount= [], [], []
    list_grape, list_class= [], []
    
    wine_list = soup.find('div', class_='ProductList-content').ul.find_all('li')

    for wine_item in wine_list:
        
        try:
            wine = wine_item.find('article', class_='ProductDisplay')

            wine_name = wine.find('div', class_='ProductDisplay-name').a['title']
            wine_link = "wine.com.br" + wine.find('div', class_='ProductDisplay-name').a['href']
            wine_country = wine.find(class_="Country").span.text
            wine_type = wine.find(class_="ProductDisplay-type").span.text
            wine_description = wine.find(class_="ProductDisplay-descriptionText").text
        
            wine_rating = wine.find('div',class_='Rating')
            wine_evaluation_tag = wine_rating.find('evaluation-tag')
            if (wine_evaluation_tag):
                wine_evaluation = float(re.compile(r"\d.\d").search(wine_evaluation_tag.prettify()).group())
                wine_rating_count = float(re.search(r"\d+", wine_rating.find(class_='Rating-count').text).group())
            else:
                wine_evaluation = 0
                wine_rating_count = 0
    
        
            price_box = wine.find(class_="ProductDisplay-priceBox")

            lowest_price = float(price_box.find(class_="Price--lowestPrice").find(class_="Price-raw").text)
            full_price = float(price_box.find(class_="Price--fullPrice").find(class_="Price-raw").text)
            discount = 1-float(lowest_price)/float(full_price)
            try:
                grape, classification = scrape_technical_details('https://' + wine_link)
            except:
                grape, classification= 'indefinido', 'indefinido'
            
        except:
            if wine_name:
                print('skip incomplete item:', wine_name)
                continue
            else:
                print("skip incomplete item: no name")
                continue
        
        list_name.append(wine_name)
        list_link.append(wine_link)
        list_country.append(wine_country)
        list_type.append(wine_type)
        list_description.append(wine_description)
        
        list_evaluation.append(wine_evaluation)
        list_rating_count.append(wine_rating_count)

        list_lowest_price.append(lowest_price)
        list_full_price.append(full_price)
        list_discount.append(discount)
        list_grape.append(grape)
        list_class.append(classification)
        
        ###print(wine_name,";", wine_evaluation, ";", wine_rating_count, ";", wine_country, ";", wine_type, ";", lowest_price, ";", full_price, ";", discount,";", wine_link)
    
    df = pd.DataFrame({
        'wine_name': list_name,
        'link': list_link,
        'country': list_country,
        'type': list_type,
        'grape': list_grape,
        'classification': list_class,
        'description': list_description,
        'evaluation': list_evaluation,
        'rating_count': list_rating_count,
        'lowest_price': list_lowest_price,
        'full_price': list_full_price,
        'discount': list_discount
    })
    
    return df
         
    

### Função Principal


In [8]:
# Módulo Principal - Processamento do Site

wine_df = pd.DataFrame(columns=['wine_name','link','country','type','grape', 'classification', 'description','evaluation','rating_count','lowest_price','full_price','discount'])

tipos_de_vinho = ['TINTO','BRANCO']

for tipo in tipos_de_vinho:

    pages_to_scrape = 4

    current_page = 1


    while (current_page != pages_to_scrape):
    
        #print("vinhos {}: Iniciando get_soup pagina {}").format(tipo, current_page)
        print('vinho ', tipo, ': Iniciando get_soup pagina ', current_page)
    
        wine_soup = get_soup(URL_BASE.format(tipo.lower(),tipo, current_page))
    
        if current_page == 1:
            pages_to_scrape = get_num_pages(wine_soup)
    
        new_df = scrape_soup(wine_soup)
        wine_df= wine_df.append(new_df, ignore_index=True)
    
        ###print(current_page)
    
        time.sleep(10)
        current_page += 1
    
print("Scrape finished.")
wine_df

vinho  TINTO : Iniciando get_soup pagina  1
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p1.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/pinhal-da-torre-special-d-o-c-tejo-2008/prod4533.html  =  <Response [200]>
Seco: Touriga Nacional (45.00%), Syrah (35.00%), Tinta Roriz (20.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/finca-la-chamiza-martin-alsina-malbec-2014/prod20028.html  =  <Response [200]>
Seco: Malbec (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/echo-de-lynch-bages-a-o-c-pauillac-2014/prod20208.html  =  <Response [200]>
Seco: Cabernet Sauvignon (73.00%), Merlot (27.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/suolo-rosso-d-o-p-primitivo-di-manduria-2016/prod20178.html  =  <Response [200]>
Meio Seco/Demi-Sec: Primitivo (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/chateau-la-fleur-


Scraping page: https://wine.com.br/vinhos/herdade-da-farizoa-grande-reserva-d-o-c-alentejo-2013/prod18185.html  =  <Response [200]>
Seco: Syrah (80%), Touriga Nacional (5%), Aragonez (15%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/la-piu-belle-2014/prod24019.html  =  <Response [200]>
Seco: Carménère, Syrah, Cabernet Sauvignon
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/pinhal-da-torre-2-worlds-2012/prod18234.html  =  <Response [200]>
Meio Seco/Demi-Sec: Syrah (35%), Touriga Nacional (35%), Tinta Roriz (30%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/marianne-floreal-2015/prod18405.html  =  <Response [200]>
Seco: Merlot (40.00%), Cabernet Sauvignon (40.00%), Shiraz (20.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/maycas-del-limari-san-julian-pinot-noir-2018/prod23933.html  =  <Response [200]>
Seco: Pinot Noir
vinho  TINTO : Iniciando get_soup pagina  6
Iniciando session_get

Scraping page: https:

vinho  TINTO : Iniciando get_soup pagina  10
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p10.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/apothic-dark-2016/prod22617.html  =  <Response [200]>
Meio Seco/Demi-Sec: Petite Sirah, Merlot, Teroldego, Cabernet Sauvignon, Zinfandel, Petit Verdot, Tannat
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/dominus-estate-napa-valley-2012/prod17247.html  =  <Response [200]>
Seco: Cabernet Sauvignon (93.00%), Petit Verdot (5.00%), Cabernet Franc (2.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/rigal-les-terrasses-a-o-c-cahors-malbec-2015/prod16959.html  =  <Response [200]>
Seco: Malbec (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/chateau-haut-brion-le-clarence-de-haut-brion-2010/prod12884.html  =  <Response [200]>
Seco: Merlot (52%), Cabernet Sauvignon (36%), Petit Verdot (2%), Cabernet F

vinho  TINTO : Iniciando get_soup pagina  16
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p16.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/tenuta-sant-antonio-dei-gigli-d-o-c-g-amarone-della-valpolicella-2013/prod19780.html  =  <Response [200]>
Meio Seco/Demi-Sec: Corvinone, Rondinella, Oseleta, Croatina, Corvina Veronese
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/tenuta-santantonio-d-o-c-g-amarone-della-valpolicella-2015/prod19799.html  =  <Response [200]>
Meio Seco/Demi-Sec: Corvina (70.00%), Croatina (5.00%), Oseleta (5.00%), Rondinella (20.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/pajaro-de-buen-aguero-d-o-p-carinena-tempranillo-2018/prod23167.html  =  <Response [200]>
Seco: Tempranillo
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/pajaro-de-buen-aguero-d-o-p-carinena-g-2018/prod23161.html  =  <Response [200]>
Seco: Ga


Scraping page: https://wine.com.br/vinhos/oxford-landing-cabernet-shiraz-2016/prod19728.html  =  <Response [200]>
Seco: Cabernet Sauvignon, Shiraz
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/bobal-desanjuan-vinas-viejas-d-o-p-utiel-requena-2016/prod19718.html  =  <Response [200]>
Seco: Bobal (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/vik-2012/prod21001.html  =  <Response [200]>
Seco: Cabernet Franc (6.00%), Cabernet Sauvignon (53.00%), Merlot (4.00%), Carménère (35.00%), Syrah (2.00%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/quinta-de-sao-joao-d-o-c-tejo-reserva-2017/prod23191.html  =  <Response [200]>
Meio Seco/Demi-Sec: Syrah, Touriga Nacional, Touriga Franca , Tinta Roriz
vinho  TINTO : Iniciando get_soup pagina  21
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p21.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/casas

...


Scraping page: https://wine.com.br/vinhos/baron-philippe-de-rothschild-pays-d-oc-syrah-2018/prod23809.html  =  <Response [200]>
Seco: Syrah
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/fortant-de-france-terroir-littoral-cabernet-sauvignon-2018/prod22775.html  =  <Response [200]>
Seco: Cabernet Sauvignon
vinho  TINTO : Iniciando get_soup pagina  41
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p41.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/baron-philippe-de-rothschild-pays-d-oc-pinot-noir-2018/prod23808.html  =  <Response [200]>
Seco: Pinot Noir
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/torre-matilde-molin-novo-i-g-t-toscana-rosso-2015-/prod18960.html  =  <Response [200]>
Seco: Sangiovese (85%), Merlot (15%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/carpineto-d-o-c-g-chianti-classico-2017/prod22072.html  =  <Response [200]>



Scraping page: https://wine.com.br/vinhos/l-esprit-de-chevalier-blanc-2014/prod19376.html  =  <Response [200]>
Seco: Sauvignon Blanc (60%), Sémillon (40%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/chateau-capitoul-oros-a-o-c-la-clape-blanc-2016/prod19916.html  =  <Response [200]>
Seco: Marsanne , Viognier, Roussanne
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/partridge-unfiltered-pinot-gris-2019/prod23275.html  =  <Response [200]>
Seco: Pinot Gris (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/gruner-veltliner-classic-white-2018/prod23115.html  =  <Response [200]>
Seco: Grüner Veltliner
vinho  BRANCO : Iniciando get_soup pagina  3
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/branco/cVINHOS-atTIPO_BRANCO-p3.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/finca-la-chamiza-chardonnay-2018/prod23057.html  =  <Response [200]>
Seco: Chardonnay (100.00%)
Ini


Scraping page: https://wine.com.br/vinhos/encostas-de-estremoz-bis-branco-2018/prod23528.html  =  <Response [200]>
Seco: Síria, Arinto, Fernão Pires
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/finca-la-daniela-torrontes-2019/prod23858.html  =  <Response [200]>
Seco: Torrontés (100.00%)
vinho  BRANCO : Iniciando get_soup pagina  8
Iniciando session_get

Scraping page: https://www.wine.com.br/vinhos/branco/cVINHOS-atTIPO_BRANCO-p8.html  =  <Response [200]>
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/finca-la-daniela-chardonnay-2019/prod23857.html  =  <Response [200]>
Seco: Chardonnay (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/cava-negra-chardonnay-375ml-2019/prod23836.html  =  <Response [200]>
Seco: Chardonnay (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/foye-selected-vineyards-chardonnay-2018/prod22679.html  =  <Response [200]>
Seco: Chardonnay (100%)
Iniciando session_get

Scraping page: 


Scraping page: https://wine.com.br/vinhos/altivo-vineyard-selection-la-rioja-torrontes-2018/prod22828.html  =  <Response [200]>
Seco: Torrontés
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/vinedo-de-los-vientos-estival-2018/prod23166.html  =  <Response [200]>
Meio Seco/Demi-Sec: Chardonnay, Moscato Bianco, Gewürztraminer
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/root-1-sauvignon-blanc-2018/prod22174.html  =  <Response [200]>
Seco: Sauvignon Blanc (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/maycas-reserva-sumaq-chardonnay-2017/prod19260.html  =  <Response [200]>
Seco: Chardonnay (100%)
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/garriguette-branco/prod22966.html  =  <Response [200]>
Seco: Grenache , Chardonnay, Bourboulenc, Clairette, Grenache Blanc
Iniciando session_get

Scraping page: https://wine.com.br/vinhos/centare-doc-grillo-sicilia-bianco-2018/prod23579.html  =  <Response [200]>
Meio Sec

,wine_name,link,country,type,grape,classification,description,evaluation,rating_count,lowest_price,full_price,discount
0,Pinhal Da Torre Special D.O.C. Tejo 2008,wine.com.br/vinhos/pinhal-da-torre-special-d-o...,Portugal,tinto,"Touriga Nacional (45.00%), Syrah (35.00%), Tin...",Seco,"Maduro, elegante, boa acidez, taninos macios, ...",5.0,1.0,826.94,999.9,0.172977
1,Finca La Chamiza Martín Alsina Malbec 2014,wine.com.br/vinhos/finca-la-chamiza-martin-als...,Argentina,tinto,Malbec (100%),Seco,"Frutado, amadeirado, corpo de médio para encor...",5.0,1.0,323.41,495.9,0.347832
2,Echo de Lynch-Bages A.O.C. Pauillac 2014,wine.com.br/vinhos/echo-de-lynch-bages-a-o-c-p...,França,tinto,"Cabernet Sauvignon (73.00%), Merlot (27.00%)",Seco,"Exuberante, encorpado, taninos presentes, boa ...",0.0,0.0,721.06,1056.9,0.317759
3,Suolo Rosso D.O.P. Primitivo di Manduria 2016,wine.com.br/vinhos/suolo-rosso-d-o-p-primitivo...,Itália,tinto,Primitivo (100%),Meio Seco/Demi-Sec,"Frutado, médio corpo, acidez agradável, tanino...",4.5,13.0,179.88,219.9,0.181992
4,Château La Fleur-Pétrus A.O.C. Pomerol 2006,wine.com.br/vinhos/chateau-la-fleur-petrus-a-o...,França,tinto,"Merlot (91%), Cabernet Franc (6%), Petit Verdo...",Seco,"Aveludado, complexo, boa acidez, corpo de médi...",0.0,0.0,3614.00,6036.9,0.401348
...,...,...,...,...,...,...,...,...,...,...,...,...
465,Dark Horse The Original Pinot Grigio 2017,wine.com.br/vinhos/dark-horse-the-original-pin...,Estados Unidos,branco,"Pinot Grigio (76.00%), Outras uvas (7.00%), Vi...",Meio Seco/Demi-Sec,"Leve, boa acidez, frutado, cítrico",4.5,26.0,84.59,106.9,0.208700
466,Salentein Killka Sauvignon Blanc 2019,wine.com.br/vinhos/salentein-killka-sauvignon-...,Argentina,branco,Sauvignon Blanc,Seco,"Suave, acidez marcada e fresco",4.5,5.0,82.24,109.9,0.251683
467,Baron Philippe de Rothschild Pays D´Oc Chardon...,wine.com.br/vinhos/baron-philippe-de-rothschil...,França,branco,Chardonnay,Seco,"Untuoso, fresco e com final saboroso",4.5,2.0,94.00,145.9,0.355723
468,Dark Horse The Original California Sauvignon B...,wine.com.br/vinhos/dark-horse-the-original-cal...,Estados Unidos,branco,Sauvignon Blanc,Meio Seco/Demi-Sec,"Boa acidez, jovem, fresco",4.5,11.0,88.12,106.9,0.175678


### Salvando o arquivo


In [9]:
wine_df.to_csv('result_wine.csv',index=False)

In [24]:
df = wine_df.groupby('classification').wine_name.count()
print(df)

classification
Meio Seco/Demi-Sec     78
Seco                  387
Suave/Doce              5
Name: wine_name, dtype: int64
